# etl

> Extraer y transformar fuentes de datos sobre delincuencia en CDMX 

In [1]:
#| default_exp etl

In [2]:
#| include: false
from nbdev.showdoc import *

In [3]:
#| export
import os
import glob
import itertools
from typing import Union
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta, date, datetime
import seaborn as sns
import requests
import h3
from shapely.geometry import Polygon, Point

In [4]:
#| export
def __get_data_path():
    " Regresa el path absoluto al direcorio de datos."
    data_name = 'datos'
    data_path = Path.cwd()
    while data_path != data_path.parent and not (data_path/data_name).exists(): data_path = data_path.parent
    if not (data_path/data_name).exists(): data_path = data_path
    return data_path/data_name
DATA_PATH = __get_data_path()
DOWNLOADS_PATH = DATA_PATH/'descargas'

## procesa_registros

* Unifica los campos NA
* Elimina records con faltantes el longitud o latitud
* Transforma en GeoDataFrame

In [5]:
#| export
def procesa_registros(records:pd.DataFrame # viene de leer datos de carpetas o víctimas
    )-> gpd.GeoDataFrame: # el mismo pero con los registros corregidos
    """Hace el procesamineto básico de los records de carpetas o víctimas."""
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    return records

## get_carpetas_from_api

In [6]:
#| export
def get_carpetas_from_api(limit:int=100 # Cuántos registros traigo del api
    )-> Union[gpd.GeoDataFrame, None]:
    """Regresa un GeoDataFrame con los primeros `limit` registros de la base abierta."""
    url = f'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=48fcb848-220c-4af0-839b-4fd8ac812c0f&limit={limit}'
    try:
        r = requests.get(url, allow_redirects=True)
    except: # Quizá deberíamos cachar cosas específicas
        return None
    records = r.json()['result']['records']
    records = pd.DataFrame(records)
    records = procesa_registros(records)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos, dayfirst=True)
    return records

In [7]:
carpetas = get_carpetas_from_api()
assert (type(carpetas) == gpd.GeoDataFrame) or (carpetas is None) 

## get_victimas_from_api

In [8]:
#| export
def get_victimas_from_api(limit=100 # Cuántos registros traigo del api
    )-> Union[gpd.GeoDataFrame, None]:
    """Regresa un GeoDataFrame con los primeros `limit` registros de la base abierta de víctimas."""
    url = f'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=d543a7b1-f8cb-439f-8a5c-e56c5479eeb5&limit={limit}'
    try:
        r = requests.get(url, allow_redirects=True)
    except:
        return None
    records = r.json()['result']['records']
    records = pd.DataFrame(records)
    records = procesa_registros(records)
    records['FechaHecho'] = pd.to_datetime(records.FechaHecho, dayfirst=True)
    records = records.rename({'FechaHecho':'fecha_hechos',
                              'Delito': 'delito',
                              'Categoria': 'categoria'}, axis=1)
    return records

In [9]:
victimas = get_victimas_from_api()
assert (type(victimas) == gpd.GeoDataFrame) or (victimas is None)

## descarga_historico_carpetas

In [10]:
#| export
def descarga_historico_carpetas()->os.path:
    """Descarga el histórico de carpetas y regrersa el path al archivo."""
    absp = os.path.abspath(os.path.join(DOWNLOADS_PATH, 'carpetas_fiscalia.csv'))
    url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/carpetas-de-investigacion-fgj-de-la-ciudad-de-mexico/carpetas_completa_febrero_2022.csv"
    if os.path.exists(absp):
        print("El archivo ya está descargado")
    else:
        r = requests.get(url, allow_redirects=True)
        open(absp, 'wb').write(r.content)
    return absp


In [11]:
pth_carpetas = descarga_historico_carpetas()
pth_carpetas

El archivo ya está descargado


'/home/plablo/git/criminologia_cdmx/datos/descargas/carpetas_fiscalia.csv'

## descarga_historico_victimas

In [12]:
#| export
def descarga_historico_victimas()->os.path:
    """Descarga el histórico de víctimas y regrersa el path al archivo."""
    absp = os.path.abspath(os.path.join(DOWNLOADS_PATH, 'victimas_carpetas_fiscalia.csv'))
    url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/victimas-en-carpetas-de-investigacion-fgj/victimas_completa_febrero_2022.csv"
    if os.path.exists(absp):
        print("El archivo ya está descargado")
    else:
        r = requests.get(url, allow_redirects=True)
        open(absp, 'wb').write(r.content)
    return absp


In [13]:
pth_victimas = descarga_historico_victimas()
pth_victimas

El archivo ya está descargado


'/home/plablo/git/criminologia_cdmx/datos/descargas/victimas_carpetas_fiscalia.csv'

## get_carpetas_desde_archivo

In [14]:
#| export
def get_carpetas_desde_archivo(archivo:os.path # El path al archivo (`descarga_historico_carpetas`)
    )->gpd.GeoDataFrame:
    """Regresa un GeoDataFrame con los registros leídos de un archivo"""
    records = pd.read_csv(archivo, low_memory=False)
    records = procesa_registros(records)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos, dayfirst=True)
    return records

In [15]:
carpetas_todas = get_carpetas_desde_archivo(pth_carpetas)
assert type(carpetas_todas) == gpd.GeoDataFrame

## get_victimas_desde_archivo

In [16]:
#| export
def get_victimas_desde_archivo(archivo # El path al archivo (`descarga_historico_victimas`)
    )->os.path:
    """Regresa un GeoDataFrame con los registros leídos de un archivo"""
    records = pd.read_csv(archivo)
    records = procesa_registros(records)
    records['FechaHecho'] = pd.to_datetime(records.FechaHecho, dayfirst=True)
    records = records.rename({'FechaHecho':'fecha_hechos',
                              'Delito': 'delito',
                              'Categoria': 'categoria'}, axis=1)
    return records

In [17]:
carpetas_todas = get_victimas_desde_archivo(pth_victimas)
assert type(carpetas_todas) == gpd.GeoDataFrame

## descarga_manzanas

In [18]:
#| export
def descarga_manzanas()->os.path:
    """ Descarga la geometría de manzanas con ids de cuadrante y colonia."""
    absp = os.path.abspath(os.path.join(DOWNLOADS_PATH, 'manzanas_identificadores.gpkg'))
    if os.path.exists(absp):
        print("El archivo ya está descargado.")
    else:
        url = "https://www.dropbox.com/s/a370kmtknhgca2y/manzanas_identificadores.gpkg?dl=1"
        r = requests.get(url, allow_redirects=True)
        open(absp, 'wb').write(r.content)
    return absp

In [19]:
pth_manzanas = descarga_manzanas()
pth_manzanas

El archivo ya está descargado.


'/home/plablo/git/criminologia_cdmx/datos/descargas/manzanas_identificadores.gpkg'

## agrega_ids_espaciales

In [20]:
#| export
def agrega_ids_espaciales(carpetas: gpd.GeoDataFrame, # Datos de carpetas o victimas (p.ej. `get_victimas_desde_archivo`)
                          metodo:str='manzanas', # manzanas/poligonos
                          tolerancia:float=500 # ¿qué tan lejos puede estar un incidente de una manzana?
    )->gpd.GeoDataFrame:
    """ Agrega ids de colonias y cuadrantes a la base de carpetas."""
    
    if 'manzana_cvegeo' in carpetas.columns:
        carpetas = carpetas.drop(columns='colonia_cve')
    if 'colonia_cve' in carpetas.columns:
        carpetas = carpetas.drop(columns='colonia_cve')
    if 'colonia_nombre' in carpetas.columns:
        carpetas = carpetas.drop(columns='colonia_nombre')
    if 'cuadrante_id' in carpetas.columns:
        carpetas = carpetas.drop(columns='cuadrante_id')
    if 'municipio_cvegeo' in carpetas.columns:
        carpetas = carpetas.drop(columns='municipio_cvegeo')
    if metodo == 'poligonos':
        shapes = os.path.join(DATA_PATH, 'criminologia_capas.gpkg')
        colonias = gpd.read_file(shapes, layer='colonias')
        cuadrantes = gpd.read_file(shapes, layer='cuadrantes')
        carpetas = (gpd.tools.sjoin(carpetas, colonias[['colonia_cve', 'colonia_nombre', 'municipio_cvegeo', 'geometry']])
                    .drop(columns=['index_right'])
                   )
        carpetas = (gpd.tools.sjoin(carpetas, cuadrantes[['cuadrante_id', 'geometry']])
                    .drop(columns=['index_right']))
    elif metodo == 'manzanas':
        crs_original = carpetas.crs
        manzanas_pth = descarga_manzanas()
        manzanas = gpd.read_file(manzanas_pth)
        manzanas['municipio_cvegeo'] = manzanas['CVE_ENT'] + manzanas['CVE_MUN']
        carpetas = (carpetas
                    .to_crs(manzanas.crs)
                    .sjoin_nearest(manzanas[['CVEGEO', 'municipio_cvegeo', 'colonia_cve', 'colonia_nombre', 
                                             'cuadrante_id', 'geometry']], max_distance=tolerancia)
                    .rename({'CVEGEO': 'manzana_cvegeo'}, axis=1)
                    .drop(columns='index_right')
                    .to_crs(crs_original))
    else:
        raise ValueError("'metodo' debe ser 'poligonos' o 'manzanas'")
    return carpetas

:::{.callout-note}

El método 'manzanas' hace un join_nearest con las manzanas que ya tienen ids de cuadrante y polígono; el método poligonos hace la unión espacial de los incidentes con las geometrías. Por lo tanto, el método `manzanas` incluye identificadores de manzanas mientras que el método `poligonos` solo agrega los identificadores de colonia y cuadrante. 

El método polígono ignora el valor de `tolerancia`

:::

In [21]:
# Pruebas método polígono
carpetas = agrega_ids_espaciales(carpetas, metodo='poligonos')
assert 'colonia_cve' in carpetas.columns
assert 'cuadrante_id' in carpetas.columns
assert 'municipio_cvegeo' in carpetas.columns
victimas = agrega_ids_espaciales(victimas, metodo='poligonos')
assert 'colonia_cve' in victimas.columns
assert 'cuadrante_id' in victimas.columns
assert 'municipio_cvegeo' in victimas.columns
# Pruebas método manzanas
carpetas = agrega_ids_espaciales(carpetas, metodo='manzanas')
assert 'colonia_cve' in carpetas.columns
assert 'cuadrante_id' in carpetas.columns
assert 'municipio_cvegeo' in carpetas.columns
assert 'manzana_cvegeo' in carpetas.columns
victimas = agrega_ids_espaciales(victimas, metodo='manzanas')
assert 'colonia_cve' in victimas.columns
assert 'cuadrante_id' in victimas.columns
assert 'municipio_cvegeo' in victimas.columns
assert 'manzana_cvegeo' in carpetas.columns
# TODO prueba de excepción

El archivo ya está descargado.
El archivo ya está descargado.


## agregar_categorias_carpetas

In [22]:
#| export
def agregar_categorias_carpetas(carpetas:gpd.GeoDataFrame, # Carpetas de investigación (p.ej. `get_carpetas_desde_archivo`) 
                                archivo_categorias:os.path # path al archivo con las categorías
                              )->gpd.GeoDataFrame:
    """Agrega una columna con categorías definidas por el usuario.

      Las categorías tienen que venir en un csv con columnas incidente y categoria que
      relacionen las categorías del usuario con la columna delitos de la base de carpetas.
    """
    if 'categoria' in carpetas.columns:
        carpetas = carpetas.drop(columns='categoria')
    if 'incidente' in carpetas.columns:
        carpetas = carpetas.drop(columns='incidente')
    categorias = pd.read_csv(archivo_categorias)
    carpetas = (carpetas
                .merge(categorias, left_on='delito', right_on='incidente', how='left')
                .drop(columns='incidente'))
    return carpetas

El archivo que relaciona categorias con carpetas:

In [23]:
categorias_carpetas = pd.read_csv(DATA_PATH/'categorias_carpetas.csv')
categorias_carpetas

,incidente,categoria
0,HOMICIDIO POR AHORCAMIENTO,Homicidios dolosos
1,HOMICIDIO POR ARMA BLANCA,Homicidios dolosos
2,HOMICIDIO POR ARMA DE FUEGO,Homicidios dolosos
3,HOMICIDIO POR GOLPES,Homicidios dolosos
4,HOMICIDIOS INTENCIONALES (OTROS),Homicidios dolosos
...,...,...
73,ROBO DE VEHICULO DE SERVICIO PÚBLICO CON VIOLE...,Robo de/en vehículo
74,ROBO DE VEHICULO DE SERVICIO PÚBLICO SIN VIOLE...,Robo de/en vehículo
75,ROBO DE VEHICULO ELECTRICO MOTOPATIN,Robo de/en vehículo
76,"OBO DE VEHICULO EN PENSION, TALLER Y AGENCIAS C/V",Robo de/en vehículo


In [24]:
carpetas = agregar_categorias_carpetas(carpetas, DATA_PATH/'categorias_carpetas.csv')
assert 'categoria' in carpetas.columns

## agregar_categorias_victimas

In [25]:
#| export
def agregar_categorias_victimas(carpetas:gpd.GeoDataFrame, # Víctimas en Carpetas de investigación (p.ej. `get_victimas_desde_archivo`)
                                archivo_categorias:os.path# path al archivo con las categorías
                                )-> gpd.GeoDataFrame:
    """Columnas con niveles definidos por el usuario

      Las categorías tienen que venir en un csv con columnas llamadas Nivel 1, Nivel 2 ...
      que relacionen los niveles con las columnas Delito y Categoría en la base de Víctimas.
    """
    columnas_nivel = [c for c in carpetas.columns if 'Nivel' in c]
    if len(columnas_nivel):
        carpetas = carpetas.drop(columns=columnas_nivel)
    categorias = pd.read_csv(archivo_categorias)
    carpetas = (carpetas
                .merge(categorias, left_on='delito', right_on='Delito', how='left')
                .rename({'categoria_x': 'categoria'}, axis=1)
                )
    return carpetas

El archivo que relaciona las categorías con las víctimas:

In [26]:
categorias_victimas = pd.read_csv(DATA_PATH/'categorias_victimas.csv')
categorias_victimas

,Delito,Categoria,Cantidad,Nivel 1,Nivel 2,Nivel 3
0,ABORTO,DELITO DE BAJO IMPACTO,168,NaN,NaN,NaN
1,ABUSO DE AUTORIDAD Y USO ILEGAL DE LA FUERZA P...,DELITO DE BAJO IMPACTO,5924,NaN,NaN,NaN
2,ABUSO DE CONFIANZA,DELITO DE BAJO IMPACTO,12050,Abuso de Confianza,NaN,NaN
3,ABUSO SEXUAL,DELITO DE BAJO IMPACTO,10238,Abuso Sexual,NaN,NaN
4,ACOSO SEXUAL,DELITO DE BAJO IMPACTO,2986,NaN,NaN,NaN
...,...,...,...,...,...,...
295,VIOLACION TUMULTUARIA,VIOLACIÓN,74,NaN,NaN,NaN
296,VIOLACION TUMULTUARIA EQUIPARADA,VIOLACIÓN,4,NaN,NaN,NaN
297,VIOLACION TUMULTUARIA EQUIPARADA POR CONOCIDO,VIOLACIÓN,2,NaN,NaN,NaN
298,VIOLACION Y ROBO DE VEHICULO,VIOLACIÓN,1,NaN,NaN,NaN


In [27]:
pth_categorias_v = os.path.join(DATA_PATH, "categorias_victimas.csv")
victimas = agregar_categorias_victimas(victimas, pth_categorias_v)
assert 'Nivel 1' in victimas.columns

## exporta_datos_visualizador

In [28]:
#| export
def exporta_datos_visualizador(carpetas:gpd.GeoDataFrame, # Datos de carpetas o victimas (p.ej. `get_victimas_desde_archivo`)
                               archivo_resultado:os.path, # A dónde exportamos el archivo
                               fecha_inicio:pd.DatetimeTZDtype=pd.to_datetime('01/01/2019'), # Dónde empezamos
                               tipo:str='victimas')->None:
    """ Escribe en archivo_resultado un csv para consumirse en el visualizador.

        La opción tipo=victimas/carpetas controla si los datos de entrada son carpetas o victimas.
    """
    columnas = ['fecha_hechos', 'delito', 'municipio_cvegeo',
                'colonia_cve', 'cuadrante_id', 'categoria', 'lat', 'long']
    if tipo == 'carpetas':
        columnas = columnas + ['categoria']
    elif tipo == 'victimas':
        columnas_nivel = [c for c in carpetas.columns if 'Nivel' in c]
        columnas = columnas + columnas_nivel
    carpetas['lat'] = carpetas.geometry.y
    carpetas['long'] = carpetas.geometry.x
    carpetas = carpetas[columnas]
    carpetas = carpetas.loc[carpetas.fecha_hechos >= fecha_inicio]
    carpetas.to_csv(archivo_resultado)

In [29]:
exporta_datos_visualizador(carpetas, "../../datos/salidas/carpetas.csv", tipo='carpetas')
exporta_datos_visualizador(victimas, "../../datos/salidas/victimas.csv", tipo='victimas')

## serie_de_tiempo_categoria

In [30]:
#| export
def serie_de_tiempo_categoria(carpetas:gpd.GeoDataFrame, # incidentes, deben traer la columna categoria (p.ej. 'agregar_categorias_carpetas')
                              fecha_inicio:pd.DatetimeTZDtype, # fecha del inicio de la serie
                              categoria, # nombre de la categoría a agregar (`agregar_categorias_carpetas`)
                              freq:str='M' # frecuencia de agregación (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)
                              )-> pd.DataFrame:
    """ Regresa una serie de tiempo con los agregados por `freq` de la `categoria`."""
    carpetas = carpetas.loc[carpetas.fecha_hechos >= fecha_inicio]
    carpetas = carpetas.loc[carpetas.categoria == categoria]
    serie = (carpetas
             .set_index('fecha_hechos')[['categoria']]
             .resample(freq)
             .size()
             .reset_index()
             .rename({0:categoria}, axis=1)
            )
    return serie

In [31]:
serie = serie_de_tiempo_categoria(carpetas, pd.to_datetime('01/01/2016'), 'Robo a pasajero')
serie

,fecha_hechos,Robo a pasajero
0,2016-01-31,1


## serie_tiempo_categorias_unidades

In [32]:
#| export
def serie_tiempo_categorias_unidades(datos:gpd.GeoDataFrame, # víctimas/carpetas, deben tener agregadas las categorías de usuario
                                     fecha_inicio:pd.DatetimeTZDtype, # fecha del inicio de la serie 
                                     tipo:str='victimas', # carpetas/victimas
                                     geografia:str='colonias',# colonias/cuadrantes
                                     freq:str='W', #frecuencia de agregación (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)
                                     categorias:list=['Nivel 1'] # lista de las categorías para agregar. Las columnas deben existir en la base
                                     )->pd.DataFrame:
    """ Regresa una serie de tiempo con los agregados por `freq` para categorias y
        la geografía especificada."""
    dummies = pd.get_dummies(datos[categorias])
    datos = datos.loc[datos.fecha_hechos >= fecha_inicio]
    datos = pd.concat([datos, dummies], axis=1)
    if geografia == 'colonias':
        id_vars = ['colonia_nombre', 'colonia_cve']
    elif geografia == 'cuadrantes':
        id_vars = ['cuadrante_id']
    else:
        return #RAISE!!!!!
    serie = (datos[['fecha_hechos', *id_vars, *list(dummies.columns)]]
             .set_index('fecha_hechos')
             .groupby([pd.Grouper(freq="M"), *id_vars])
             .sum())
    serie = serie.reset_index().melt(id_vars=['fecha_hechos', *id_vars])
    return serie

In [33]:
serie = serie_tiempo_categorias_unidades(victimas, pd.to_datetime('01/01/2019'))
serie

,fecha_hechos,colonia_nombre,colonia_cve,variable,value
0,2019-01-31,7 DE JULIO,868.0,Nivel 1_Abuso de Confianza,0
1,2019-01-31,AGRICOLA ORIENTAL V,885.0,Nivel 1_Abuso de Confianza,0
2,2019-01-31,ASTURIAS (AMPL),170.0,Nivel 1_Abuso de Confianza,0
3,2019-01-31,AÑO DE JUAREZ,1469.0,Nivel 1_Abuso de Confianza,0
4,2019-01-31,"BOSQUE DE CHAPULTEPEC I, II Y III SECCIONES",187.0,Nivel 1_Abuso de Confianza,0
...,...,...,...,...,...
683,2019-01-31,TACUBAYA,181.0,Nivel 1_Violencia Familiar,0
684,2019-01-31,TENORIOS,1401.0,Nivel 1_Violencia Familiar,0
685,2019-01-31,TEPETATAL,14.0,Nivel 1_Violencia Familiar,0
686,2019-01-31,TLAXPANA,204.0,Nivel 1_Violencia Familiar,0


## punto_to_hexid

In [34]:
#| export
def punto_to_hexid(punto:Point, # Punto para obtener el hex_id
                  resolution: int # Escala de H3
                  ):
    """Regresa el hexid (h3) del punto."""
    return h3.geo_to_h3(punto.y, punto.x, resolution)

## agrega_en_hexagonos

In [35]:
#| export
def agrega_en_hexagonos(puntos:gpd.GeoDataFrame, # Los datos que vamos a pasar a hexágonos
                       resolution:int # Escala de H3
                       ):
    """Regresa un GeoDataFrame con las cuentas de puntos agregadas en hexágonos."""
    puntos.loc[:,'hex_id'] = puntos.loc[:,'geometry'].apply(punto_to_hexid, args=[resolution])
    by_hex = puntos.groupby('hex_id').size().reset_index()
    # by_hex['geometry'] = by_hex['hex_id'].apply(lambda hex_id: Polygon(h3.h3_to_geo_boundary(hex_id)))
    by_hex['geometry'] = by_hex['hex_id'].apply(lambda hex_id: Polygon([x[::-1] for x in h3.h3_to_geo_boundary(hex_id)]))
    by_hex = gpd.GeoDataFrame(by_hex).rename({0:'incidentes'}, axis=1).set_crs(epsg=4326)
    return by_hex

In [36]:
#| hide
import nbdev; nbdev.nbdev_export()